##### Read the CSV file using the spark dataframe reader

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
races_schema = StructType(fields=[StructField("raceID",IntegerType(), False),
                                  StructField("year",IntegerType(), False),
                                  StructField("round", IntegerType(), True),
                                  StructField("circuitId", IntegerType(), True),
                                  StructField("name", StringType(), True),
                                  StructField("date", DateType(), True),
                                  StructField("time", StringType(), True),
                                  StructField("url", StringType(), True)])

In [0]:
races_df = spark.read \
.option("header", True) \
.schema(races_schema) \
.csv(f"{raw_folder_path}/{v_file_date}/races.csv")

##### Add ingestion date and race_timestamp to the dataframe

In [0]:
races_with_ingestion_date_df = add_ingestion_date(races_df)

In [0]:
from pyspark.sql.functions import to_timestamp, concat, col, lit

In [0]:
races_with_timestamp_df = races_with_ingestion_date_df.withColumn("race_timestamp", to_timestamp(concat(col('date'), lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss'))\
    .withColumn("file_date", lit(v_file_date))

##### Select only the columns required & rename as required

In [0]:
races_selected_df = races_with_timestamp_df.select(col('raceId').alias('race_id'), col('year').alias('race_year'), 
                                                   col('round'), col('circuitId').alias('circuit_id'),col('name'), col('ingestion_date'), col('race_timestamp'))

##### Write the output to processed container in delta format

In [0]:
races_selected_df.write.mode("overwrite").partitionBy("race_year").format("delta").saveAsTable("f1_processed.races")